In [ ]:
!pip install pmaw==3.0.0

In [ ]:
from pmaw import PushshiftAPI

api = PushshiftAPI()

In [ ]:
import praw

reddit = praw.Reddit(client_id = "###", client_secret="###", user_agent="###")

In [ ]:
import datetime as dt
from tqdm import tqdm
import pandas as pd

def filter_list_of_dicts(dicts):
    ids = set()
    for d in dicts:
        if d['id'] not in ids:
            ids.add(d['id'])
            yield d

# starting from 1/1/2014 to 1/1/2023 call api to get posts for each week
start_epoch=int(dt.datetime(2015, 1, 1).timestamp())
end_epoch=int(dt.datetime(2021, 1, 1).timestamp())

post_list = []
counter = 0
for current_epoch in tqdm(range(start_epoch, end_epoch, 604800)):
    posts = api.search_submissions(subreddit="IAmA", limit=None, after=current_epoch, before=current_epoch+604800)
    # post_list = [post for post in posts]
    new_posts = [post for post in posts if post['num_comments'] > 1000]
    # eliminate duplicates in new_posts by checking id
    new_posts = list(filter_list_of_dicts(new_posts))


    for i, post in enumerate(new_posts):
        print(f'{counter}: {post["title"]}')
        counter += 1
        submission = reddit.submission(id=post['id'])
        submission.comments.replace_more(limit=0)
        # comment_list = [comment.body for comment in submission.comments.list()]
        comment_list = [{'body': comment.body, 'score': comment.score} for comment in submission.comments.list()]
            
        new_posts[i]['comments'] = comment_list

    post_list.extend(new_posts)
    df = pd.DataFrame(post_list)
    df.to_csv('data_2015_to_2021.csv', index=False)
